# Hướng dẫn chuyển đổi MRI sang CT tổng hợp

Notebook này hướng dẫn cách sử dụng mô hình CycleGAN đã được huấn luyện để chuyển đổi ảnh MRI sang ảnh CT tổng hợp cho mục đích xạ trị.

## Nội dung
1. Thiết lập môi trường
2. Tải mô hình đã huấn luyện
3. Chuẩn bị dữ liệu MRI
4. Chuyển đổi MRI sang CT tổng hợp
5. Hiển thị kết quả
6. Phân đoạn mô dựa trên đơn vị Hounsfield (HU)
7. Lưu kết quả
8. Đánh giá chất lượng chuyển đổi (tùy chọn)

## 1. Thiết lập môi trường

Đầu tiên, chúng ta cần cài đặt và nhập các thư viện cần thiết.

In [ ]:
# Cài đặt các thư viện cần thiết (nếu chưa có)
# !pip install torch torchvision numpy matplotlib pydicom nibabel SimpleITK scikit-image pyyaml

import os
import sys
import yaml
import numpy as np
import matplotlib.pyplot as plt
import torch
from pathlib import Path

# Thêm thư mục gốc vào đường dẫn để có thể nhập các module
sys.path.append('..')

# Nhập các module cần thiết từ dự án
from models.cycle_gan import CycleGANModel
from utils.data_utils import load_dicom_series, save_dicom_series, normalize_image
from utils.visualization import display_slices, compare_images
from utils.config import load_config

# Kiểm tra GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Sử dụng thiết bị: {device}")

## 2. Tải cấu hình và mô hình đã huấn luyện

Tiếp theo, chúng ta sẽ tải cấu hình và mô hình CycleGAN đã được huấn luyện.

In [ ]:
# Tải cấu hình
config_path = '../configs/default.yaml'
config = load_config(config_path)

# Đường dẫn đến checkpoint mô hình
checkpoint_path = '../data/output/models/checkpoints/best_model.pth'

# Khởi tạo mô hình
model = CycleGANModel(
    input_channels=config['model']['input_channels'],
    output_channels=config['model']['output_channels'],
    generator_filters=config['model']['generator_filters'],
    discriminator_filters=config['model']['discriminator_filters'],
    n_residual_blocks=config['model']['n_residual_blocks']
)

# Tải trọng số mô hình
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Đã tải mô hình từ epoch {checkpoint['epoch']}")
else:
    print(f"Không tìm thấy checkpoint tại {checkpoint_path}")
    print("Vui lòng huấn luyện mô hình trước hoặc tải mô hình đã huấn luyện")

model.to(device)
model.eval()  # Đặt mô hình ở chế độ đánh giá

## 3. Chuẩn bị dữ liệu MRI

Bây giờ chúng ta sẽ tải và chuẩn bị dữ liệu MRI để chuyển đổi.

In [ ]:
# Đường dẫn đến thư mục chứa ảnh MRI DICOM
mri_dir = '../data/test/mri'

# Tải dữ liệu MRI
mri_volume, mri_metadata = load_dicom_series(mri_dir)
print(f"Kích thước khối MRI: {mri_volume.shape}")

# Chuẩn hóa dữ liệu MRI
mri_normalized = normalize_image(mri_volume)

# Hiển thị một số lát cắt MRI
middle_slice = mri_volume.shape[0] // 2
display_slices(mri_normalized, start_slice=middle_slice-5, end_slice=middle_slice+6, step=2, 
               title="Lát cắt MRI", cmap="gray")

## 4. Chuyển đổi MRI sang CT tổng hợp

Sử dụng mô hình CycleGAN để chuyển đổi ảnh MRI sang ảnh CT tổng hợp.

In [ ]:
# Hàm để xử lý từng lát cắt và tạo CT tổng hợp
def generate_synthetic_ct(mri_volume, model, device, batch_size=4):
    model.eval()
    synthetic_ct = np.zeros_like(mri_volume)
    
    with torch.no_grad():
        for i in range(0, mri_volume.shape[0], batch_size):
            batch_end = min(i + batch_size, mri_volume.shape[0])
            batch = mri_volume[i:batch_end]
            
            # Chuyển đổi sang tensor và thêm kênh
            batch_tensor = torch.from_numpy(batch).float().unsqueeze(1).to(device)
            
            # Tạo CT tổng hợp
            synthetic_batch = model.generator_MRI_to_CT(batch_tensor)
            
            # Chuyển về numpy và loại bỏ kênh
            synthetic_batch = synthetic_batch.cpu().numpy().squeeze(1)
            synthetic_ct[i:batch_end] = synthetic_batch
            
            print(f"Đã xử lý {batch_end}/{mri_volume.shape[0]} lát cắt", end='\r')
    
    print(f"\nĐã hoàn thành chuyển đổi {mri_volume.shape[0]} lát cắt MRI sang CT tổng hợp")
    return synthetic_ct

# Tạo CT tổng hợp từ MRI
synthetic_ct = generate_synthetic_ct(mri_normalized, model, device)

# Chuyển đổi giá trị pixel sang đơn vị Hounsfield (HU)
# Giả sử mô hình tạo ra giá trị trong khoảng [-1, 1] cần được ánh xạ sang khoảng HU thích hợp
hu_min, hu_max = -1000, 3000  # Khoảng HU điển hình
synthetic_ct_hu = (synthetic_ct + 1) / 2 * (hu_max - hu_min) + hu_min

## 5. Hiển thị kết quả

So sánh ảnh MRI gốc với ảnh CT tổng hợp.

In [ ]:
# Hiển thị một số lát cắt CT tổng hợp
middle_slice = synthetic_ct_hu.shape[0] // 2
display_slices(synthetic_ct_hu, start_slice=middle_slice-5, end_slice=middle_slice+6, step=2, 
               title="Lát cắt CT tổng hợp", cmap="gray", vmin=-200, vmax=400)

# So sánh MRI và CT tổng hợp
for slice_idx in range(middle_slice-4, middle_slice+5, 2):
    compare_images(
        mri_normalized[slice_idx], synthetic_ct_hu[slice_idx],
        titles=[f"MRI (lát cắt {slice_idx})", f"CT tổng hợp (lát cắt {slice_idx})"],
        cmaps=["gray", "gray"],
        vmin_vmax=[(None, None), (-200, 400)]
    )

## 6. Phân đoạn mô dựa trên đơn vị Hounsfield (HU)

Phân đoạn các loại mô khác nhau dựa trên giá trị HU của chúng.

In [ ]:
# Định nghĩa các khoảng HU cho các loại mô khác nhau
tissue_ranges = {
    'Không khí': (-1000, -950),
    'Phổi': (-950, -700),
    'Mỡ': (-700, -100),
    'Nước/Mô mềm': (-100, 100),
    'Xương': (100, 3000)
}

# Tạo mặt nạ cho từng loại mô
tissue_masks = {}
for tissue, (min_hu, max_hu) in tissue_ranges.items():
    mask = (synthetic_ct_hu >= min_hu) & (synthetic_ct_hu < max_hu)
    tissue_masks[tissue] = mask

# Hiển thị phân đoạn mô cho một lát cắt
def display_tissue_segmentation(ct_slice, masks, slice_idx):
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    # Hiển thị CT gốc
    axes[0].imshow(ct_slice, cmap='gray', vmin=-200, vmax=400)
    axes[0].set_title(f"CT tổng hợp (lát cắt {slice_idx})")
    axes[0].axis('off')
    
    # Hiển thị từng loại mô
    for i, (tissue, mask) in enumerate(masks.items(), 1):
        if i < len(axes):
            axes[i].imshow(mask[slice_idx], cmap='viridis')
            axes[i].set_title(f"{tissue}")
            axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Hiển thị phân đoạn mô cho lát cắt giữa
display_tissue_segmentation(synthetic_ct_hu[middle_slice], tissue_masks, middle_slice)

## 7. Lưu kết quả

Lưu ảnh CT tổng hợp dưới dạng tệp DICOM.

In [ ]:
# Tạo thư mục đầu ra
output_dir = '../data/output/notebook_demo'
os.makedirs(output_dir, exist_ok=True)

# Lưu CT tổng hợp dưới dạng tệp DICOM
save_dicom_series(
    synthetic_ct_hu,
    output_dir,
    reference_metadata=mri_metadata,
    modality='CT',
    series_description='Synthetic CT from MRI',
    window_center=40,  # Giá trị cửa sổ phù hợp cho CT
    window_width=400   # Độ rộng cửa sổ phù hợp cho CT
)

print(f"Đã lưu CT tổng hợp tại: {output_dir}")

## 8. Đánh giá chất lượng chuyển đổi (tùy chọn)

Nếu có sẵn dữ liệu CT thực, chúng ta có thể đánh giá chất lượng của CT tổng hợp.

In [ ]:
# Kiểm tra xem có dữ liệu CT thực hay không
real_ct_dir = '../data/test/ct'
if os.path.exists(real_ct_dir):
    # Tải dữ liệu CT thực
    real_ct, real_ct_metadata = load_dicom_series(real_ct_dir)
    print(f"Kích thước khối CT thực: {real_ct.shape}")
    
    # Đảm bảo CT thực và CT tổng hợp có cùng kích thước
    if real_ct.shape == synthetic_ct_hu.shape:
        # Tính toán các chỉ số đánh giá
        from utils.metrics import calculate_mae, calculate_psnr, calculate_ssim
        
        mae = calculate_mae(real_ct, synthetic_ct_hu)
        psnr = calculate_psnr(real_ct, synthetic_ct_hu)
        ssim = calculate_ssim(real_ct, synthetic_ct_hu)
        
        print(f"Đánh giá chất lượng CT tổng hợp:")
        print(f"MAE: {mae:.2f} HU")
        print(f"PSNR: {psnr:.2f} dB")
        print(f"SSIM: {ssim:.4f}")
        
        # So sánh CT thực và CT tổng hợp
        for slice_idx in range(middle_slice-4, middle_slice+5, 2):
            compare_images(
                real_ct[slice_idx], synthetic_ct_hu[slice_idx],
                titles=[f"CT thực (lát cắt {slice_idx})", f"CT tổng hợp (lát cắt {slice_idx})"],
                cmaps=["gray", "gray"],
                vmin_vmax=[(-200, 400), (-200, 400)]
            )
            
            # Hiển thị bản đồ sai số
            error_map = np.abs(real_ct[slice_idx] - synthetic_ct_hu[slice_idx])
            plt.figure(figsize=(8, 6))
            plt.imshow(error_map, cmap='hot', vmin=0, vmax=200)
            plt.colorbar(label='Sai số tuyệt đối (HU)')
            plt.title(f"Bản đồ sai số (lát cắt {slice_idx})")
            plt.axis('off')
            plt.show()
    else:
        print(f"Kích thước không khớp: CT thực {real_ct.shape} vs CT tổng hợp {synthetic_ct_hu.shape}")
else:
    print(f"Không tìm thấy dữ liệu CT thực tại {real_ct_dir}")
    print("Bỏ qua bước đánh giá chất lượng")

## Kết luận

Trong notebook này, chúng ta đã thực hiện các bước sau:
1. Tải mô hình CycleGAN đã được huấn luyện
2. Chuẩn bị dữ liệu MRI đầu vào
3. Chuyển đổi MRI sang CT tổng hợp
4. Hiển thị và so sánh kết quả
5. Phân đoạn các loại mô dựa trên giá trị HU
6. Lưu kết quả dưới dạng tệp DICOM
7. Đánh giá chất lượng chuyển đổi (nếu có dữ liệu CT thực)

CT tổng hợp từ MRI có thể được sử dụng trong lập kế hoạch xạ trị, giúp giảm liều lượng bức xạ cho bệnh nhân và cải thiện quy trình lâm sàng.